<a href="https://colab.research.google.com/github/Katlego-TheDataScientist/attendence_backend/blob/main/backend_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>